In [ ]:
!pip install torchinfo

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import models
import os
from typing import List, Any, Tuple
from torchinfo import summary
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
classification_models = models.list_models(module=models)

print(len(classification_models), "classification models:", classification_models)

In [ ]:
efficientnet_v2_m = models.efficientnet_v2_m(weights=None)

In [ ]:
dict(efficientnet_v2_m.named_children()).keys()

In [ ]:
efficientnet_v2_m

In [ ]:
class CarTypeClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CarTypeClassifier, self).__init__()
        self.model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        #self.num_classes = 10 #Number of classes
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes) #Change classification head

        for param in self.model.parameters():
            param.requires_grad = False

        for i in range(7, 9): #Retrain the last two blocks in features
            for param in self.model.features[i].parameters():
                param.requires_grad = True
            

    def forward(self, x):
        return self.model(x)


In [ ]:
batch_size=64
IMG_SIZE = (384, 384)

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomAutocontrast(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE),
    ])

dataset_train = ImageFolder(
    '/kaggle/input/carsimages/cars/train',
    transform=train_transforms,
    )

dataloader_train = DataLoader(
    dataset_train,
    shuffle=True,
    batch_size=batch_size
    )

In [ ]:
val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE),
    ])

dataset_val = ImageFolder(
    '/kaggle/input/carsimages/cars/val',
    transform=val_transforms,
    )

dataloader_val = DataLoader(
    dataset_val,
    batch_size=batch_size
    )

In [ ]:
num_classes = 10
model = CarTypeClassifier(num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
lr_schedule = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
summary(model)

In [ ]:
num_epochs = 10

In [ ]:
val_losses=[]
val_accs=[]

model.train()
for epoch in range(num_epochs):
    train_loss = list()
    for data, labels in tqdm(dataloader_train):
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        preds = model(data)
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    #Validation
    model.eval()
    running_loss=0
    correct=0
    total=0

    with torch.no_grad():
        for data, labels in tqdm(dataloader_val):
            data, labels = data.to(device), labels.to(device)
            preds=model(data)
            loss= criterion(preds,labels)
            running_loss+=loss.item()
            _, predicted = preds.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss=running_loss/len(dataloader_val)
    val_acc=100.*correct/total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch {epoch+1} \t\t Training Loss: {torch.tensor(train_loss).mean():.5f}  \t\t Validation Loss: {torch.tensor(val_loss).mean():.5f} \t\t Validation Accuracy: {torch.tensor(val_acc).mean():.5f}')


In [ ]:
torch.save(model, 'cars_model.pt')